In [53]:
import angr
p = angr.Project('/Users/jglez2330/Library/Mobile Documents/com~apple~CloudDocs/personal/STARK-attesttation/test/a.out', load_options={'auto_load_libs': False})

cfg = p.analyses.CFGFast()

print("This is the graph:", cfg.graph)
print("It has %d nodes and %d edges" % (len(cfg.graph.nodes()), len(cfg.graph.edges())))

entry_node = cfg.get_any_node(p.entry)

print("There were %d contexts for the entry block" % len(cfg.get_all_nodes(p.entry)))
print("Predecessors of the entry point:", entry_node.predecessors)
print("Successors of the entry point:", entry_node.successors)
print("Successors (and type of jump) of the entry point:", [ jumpkind + " to " + str(node.addr) for node,jumpkind in cfg.get_successors_and_jumpkind(entry_node) ])

WARNING  | 2024-10-09 08:42:22,592 | cle.backends.macho.macho | The Mach-O backend is not well-supported. Good luck!
WARNING  | 2024-10-09 08:42:22,597 | cle.backends.macho.macho | 14 is not handled yet
WARNING  | 2024-10-09 08:42:22,598 | cle.backends.macho.macho | 27 is not handled yet
WARNING  | 2024-10-09 08:42:22,599 | cle.backends.macho.macho | 42 is not handled yet


This is the graph: DiGraph with 14 nodes and 15 edges
It has 14 nodes and 15 edges
There were 1 contexts for the entry block
Predecessors of the entry point: []
Successors of the entry point: [<CFGNode 0x100000f52[6]>]
Successors (and type of jump) of the entry point: ['Ijk_Call to 4294971218']


In [54]:
from hashlib import sha256

In [55]:
# Define the graph as an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': ['E'],
    'D': [],
    'E': ['F'],
    'F': []
}

# DFS function to find the path from start to end
def dfs(graph, start, goal, path=None):
    if path is None:
        path = []

    path = path + [start]

    if start == goal:
        return path

    if start not in graph:
        return None

    for node in graph[start]:
        if node not in path:
            new_path = dfs(graph, node, goal, path)
            if new_path:
                return new_path

    return None

# Function to print the path from one node to another
def find_and_print_path(graph, start, goal):
    path = dfs(graph, start, goal)
    if path:
        print("Path found:", " -> ".join(path))
        return path
    else:
        print(f"No path found from {start} to {goal}")
        return []

# Example: find a path from node 'A' to node 'F'
find_and_print_path(graph, 'A', 'F')


Path found: A -> C -> E -> F


['A', 'C', 'E', 'F']

In [56]:
path = find_and_print_path(graph, 'A', 'F')
path = ['A', 'C','E', 'F']
execution = [sha256(i.encode("UTF-8")) for i in path]

trace = []
nonce = 1
trace.append(sha256(str(nonce).encode("UTF-8")))
for i in range(len(execution)-1):
    trace.append(sha256(execution[i].digest() + trace[-1].digest()))
trace = [int.from_bytes(i.digest()) for i in trace]
trace

Path found: A -> C -> E -> F


[48635463943209834798109814161294753926839975257569795305637098542720658922315,
 2542819216453761409254500233514721456325805441121836427145960347314070197973,
 3066898672772311992480761421518869773108402208945700242704374602155849533315,
 18387883697599163565981562079465157039772126126815288883976149418505564317327]

In [57]:
import math
def next_power_of_2(n):
    if n < 1:
        return 1
    return 2 ** math.ceil(math.log2(n))

In [58]:
from Field import  *
from Polynomial import  *
exp_factor = 8
root_pow = next_power_of_2(len(trace))
field = Field.main()
g = field.primitive_nth_root(root_pow)
h = field.primitive_nth_root(root_pow*exp_factor)

G = [g^i for i in range(len(trace))]
H = [h^i for i in range(root_pow*exp_factor)]


In [59]:
trace = [FieldElement(i, Field.main()) for i in trace]

In [60]:
fx = Polynomial.interpolate_domain(G, trace)
fx

In [61]:
random_w = FieldElement(4, Field.main())
H_w = [h*random_w for h in H]
fx_exp_eval = fx.evaluate_domain(H_w)


In [62]:
from Merkle import *
print(len(fx_exp_eval))
fx_root =  MerkleTree(fx_exp_eval)
fx_root

32


# Contrains
3 constrains
final and initial value must be the same
and intermidiate value must be the following formula:
$a_n = a_{n-1} + m$
where m is the slope of the line

In [63]:
nonce_hash = FieldElement(int.from_bytes(sha256(str(nonce).encode("UTF-8")).digest()), Field.main())
p0_num = fx - Polynomial([nonce_hash])
x = Polynomial([field.zero(), field.one()])
p0_dem = x - Polynomial([G[0]])
p0 = p0_num/p0_dem
p0

In [64]:
p1_num = fx - Polynomial([FieldElement(18387883697599163565981562079465157039772126126815288883976149418505564317327, Field.main())])
x = Polynomial([field.zero(), field.one()])
p1_dem = x - Polynomial([g^(len(trace)-1)])
p1 = p1_num/p1_dem
p1

In [65]:
cp = p0 + p1
cp_eval = cp.evaluate_domain(H_w)
for i in cp_eval:
    print(i.value)
mk = MerkleTree(cp_eval)

57823405364060886188555359041969056157
140849091095845268835812263872059931102
231185967969200169823088652554244185155
4641813523158314555951435404703315045
124942105743773737960584479743119366950
178368107505163824085659798219169922050
99304909021586273782227702243077774899
17585371476462611796730778114375964463
80079697097430960573689917370268640949
90011730102390705426501427309886522619
175625601643402208309249644535033276652
173762096990047321033980517260126951283
223902939227968640832418215271914056402
216173103704728560000542015103239768024
222382053933809986941908217580760094285
198312685156258749042562534562842288082
200165093596834946061552425291530790674
206604390973088396649627454456954478088
240823542924049396907298594164356548555
246442998876634487612771969028369544793
102465413000934157671574942388258511662
132374903102926308069208466744283764216
102633587629264630217117546619506700916
105191048606097916444300976986225729194
114293590736694876306819447147860841912
1489657

# FRI

In [66]:
from FRI import *

fri = Fri(H_w)

fri_polys, fri_domains, fri_layers, fri_merkles = fri.commit(cp, H_w, cp_eval, mk, None)

next_len fri layer: 16
next_len fri layer: 8


In [67]:
#fri.decommit(1, fri_polys, fri_domains, fri_layers, fri_merkles)
[len(i) for i in fri_layers]

[32, 16, 8]

# Verify




In [68]:
mt_root = fx_root 

In [69]:
# 3 cp so 3 alphas
alpha0, alpha1,alpha2 = FieldElement(1, Field.main()), FieldElement(1, Field.main()), FieldElement(1, Field.main())

In [70]:
# CP merkle root
cp_root = mk

In [71]:
idx = 1

In [72]:
proof_f = fri.decommit_on_query(idx, fx_exp_eval, mt_root)

In [73]:
proof_cp = fri.decommit_on_fri(idx, fri_polys, fri_domains, fri_layers, fri_merkles)


In [74]:
# Basic check
assert len(proof_cp) % 2
v_last_one = proof_cp.pop()

In [75]:
last_one = fri_polys[-1].coefficients[0] # poly 0 shall be constant

v_last_one
assert v_last_one == last_one


In [76]:
from Merkle import verify_decommitment

In [77]:
# Check polynomial constraint
# First check evaluation of polynomial f is honest
v_f_eval = proof_f[0::2]
v_f_auth = proof_f[1::2]
assert len(v_f_eval) == len(v_f_auth)
for i in range(len(v_f_eval)):
    assert verify_decommitment(idx + 8 * i, v_f_eval[i], v_f_auth[i], mt_root.root ), f'in iter {i}, go wrong'
# Then check whether they could satify the recursive condition
v_cp0 = proof_cp[:2]
length = len(fri_layers[0]) # 8192 shoud be a prior knowledge
assert verify_decommitment(idx % length, v_cp0[0], v_cp0[1], cp_root.root)

6c6038599b451ddf2a65e0b0a02b92546a6a47c35271508ab78a411c6d52994c
6c6038599b451ddf2a65e0b0a02b92546a6a47c35271508ab78a411c6d52994c
7599e83a45120e92505e0cd4b221567df58b3cb6f7637dbd75c8c88bc718e3c4


In [78]:
# Check the polynomial constraint
fx = v_f_eval[0]
fgx = v_f_eval[1]
x = H_w[idx]
p0 = (fx - nonce_hash) / (x - G[0])
p1 = (fx - FieldElement(18387883697599163565981562079465157039772126126815288883976149418505564317327, Field.main()))/ (x - G[len(trace)-1])

#p2 = p2_num/p2_dem
assert v_cp0[0] == (p0 + p1)
print("poly constraint success!")

poly constraint success!


In [79]:
# Check low degree
v_cp = proof_cp[::2]
v_auth = proof_cp[1::2]
assert len(v_cp) == len(v_auth)
assert len(v_cp) % 2 == 0
k = length
for i in range(len(v_cp)//2 -1):
    iter_idx = idx % k
    iter_sib_idx = (idx + k // 2) % k
    assert verify_decommitment(iter_idx, v_cp[2*i], v_auth[2*i], fri_merkles[i].root)
    assert verify_decommitment(iter_sib_idx, v_cp[(2*i) + 1], v_auth[(2*i)+1], fri_merkles[i].root)
    k = k // 2

7599e83a45120e92505e0cd4b221567df58b3cb6f7637dbd75c8c88bc718e3c4
7599e83a45120e92505e0cd4b221567df58b3cb6f7637dbd75c8c88bc718e3c4


In [82]:
k = length
x = H_w[idx]
beta = [FieldElement(1, Field.main()) for i in range(99)]
for i in range(len(v_cp)//2 - 1):
    op1 = (v_cp[2*i] + v_cp[2*i + 1]) / FieldElement(2, Field.main())
    op2 = (v_cp[2*i] - v_cp[2*i + 1]) / (FieldElement(2, Field.main())*x)
    rhs = op1 + beta[i] * op2
    assert v_cp[2*(i+1)] == rhs, f" round {i}, CP(i+1) is {v_cp[2*(i+1)]} while rhs is {rhs}"
    x = x^2
print("low degree test success!")

low degree test success!
